In [1]:
# Download nltk packages
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt")
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpor

In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer # for vectorizing text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances # calculate similarity between vectors
from nltk.tokenize import word_tokenize# Import word tokenizer from nltk
from nltk import pos_tag # import pos tagger from nltk
from nltk.corpus import stopwords # get the stopwords list from the nltk package
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer # lemmatize the tokens
from sklearn import linear_model 
from sklearn import naive_bayes
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learn-ai-bbc/BBC News Train.csv
/kaggle/input/learn-ai-bbc/BBC News Sample Solution.csv
/kaggle/input/learn-ai-bbc/BBC News Test.csv


In [3]:
# Check the shape of train and test data
train_ = pd.read_csv('/kaggle/input/learn-ai-bbc/BBC News Train.csv')
test_ = pd.read_csv('/kaggle/input/learn-ai-bbc/BBC News Test.csv')

In [4]:
# Display first five rows of train data
train_.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [5]:
test_.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [6]:
print(f"train_ data has {train_.shape[0]} rows and {train_.shape[1]} columns")
print(f"test_ data has {test_.shape[0]} rows and {test_.shape[1]} columns")

train_ data has 1490 rows and 3 columns
test_ data has 735 rows and 2 columns


In [7]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB


In [8]:
test_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  735 non-null    int64 
 1   Text       735 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.6+ KB


In [9]:
train_['Category'].value_counts(normalize = True) * 100

sport            23.221477
business         22.550336
politics         18.389262
entertainment    18.322148
tech             17.516779
Name: Category, dtype: float64

All classes seem to be more or less balanced, no major class imbalance appears.

In [10]:
# Inspect the first row of train data
train_.iloc[0]

ArticleId                                                 1833
Text         worldcom ex-boss launches defence lawyers defe...
Category                                              business
Name: 0, dtype: object

In [11]:
# display text in first row of train data
train_.iloc[:5]['Text']

0    worldcom ex-boss launches defence lawyers defe...
1    german business confidence slides german busin...
2    bbc poll indicates economic gloom citizens in ...
3    lifestyle  governs mobile choice  faster  bett...
4    enron bosses in $168m payout eighteen former e...
Name: Text, dtype: object

### Use CountVectorizer technique

- Let's use the most basic Bag of Words technique CountVectorizer for vectorizing the data which creates huge sparse matrix that stores count of all words in our corpus.

- And Logistic Regression as our baseline model to create our benchmark score.

- Since the competition used Accuracy as the metric, we will use Accuracy to evaluate performance of the model.

In [12]:
# Use KFold cross validation to evaluate clasifier's performance
def kf_cv(train_, model, vectorizer, stop_words = None):
    accuracy_scores = []
    
    # read the training data
    train_ = pd.read_csv('/kaggle/input/learn-ai-bbc/BBC News Train.csv')
    
    # Encode target feature
    target_map = {'sport': 0, 'business': 1, 'entertainment': 2, 'tech': 3, 'politics': 4}
    train_['Category'] = train_['Category'].map(target_map)
    print(train_['Category'].value_counts())
    
    # Create a new column called kfold and fill it with -1
    train_['kfold'] = -1
    
    # randomize the rows of the data
    train_ = train_.sample(frac = 1).reset_index(drop = True)
    
    # fetch labels
    y = train_['Category'].values
    
    # Initiate the kfold class
    kf = model_selection.KFold(n_splits = 5)
    
    # fill the new kfold column
    for f, (t_, v_) in enumerate(kf.split(X = train_, y = y)):
        train_.loc[v_, 'kfold'] = f
        
    # go over the folds created
    for fold_ in range(5):
        # create train and validation dataframes using folds
        train_df = train_[train_.kfold != fold_].reset_index(drop = True)
        valid_df = train_[train_.kfold == fold_].reset_index(drop = True)
        
        # apply preprocessing to the text column
        # -- lowercase text 
        train_df['Text'] = train_df['Text'].apply(lambda x : x.lower())
        valid_df['Text'] = valid_df['Text'].apply(lambda x : x.lower())
        # -- tokenize text
        train_df['Text'] = train_df['Text'].apply(lambda x : " ".join(word_tokenize(x)))
        valid_df['Text'] = valid_df['Text'].apply(lambda x : " ".join(word_tokenize(x)))
        # stop words removal -- optional
        if stop_words:
            print('Stop words removed')
            #stop_words = set(stopwords.words('english'))
            train_df['Text'] = train_df['Text'].apply(lambda x : " ".join([word for word in x.split() if word not in stop_words]))
            valid_df['Text'] = valid_df['Text'].apply(lambda x : " ".join([word for word in x.split() if word not in stop_words]))

        # Initialize vectorizer
        vec = vectorizer

        # fit vec on training data text
        vec.fit(train_df['Text'])

        # transform training and validation data text
        xtrain = vec.transform(train_df['Text'])
        xval = vec.transform(valid_df['Text'])
        print(f"Shape of xtrain: {xtrain.shape}")

        # Initialize pipeline
        model = model

        # fit the model on Text and Category
        model.fit(xtrain, train_df.Category)

        # make predictions on validation set
        preds = model.predict(xval)

        # calculate accuracy
        accuracy = metrics.accuracy_score(valid_df.Category, preds)
        accuracy_scores.append(accuracy)
        
        print(f"Fold: {fold_}")
        print(f"Accuracy = {accuracy}")
        print("")
        
    print(f"Mean accuracy score: {np.mean(accuracy_scores)}")
    print(f"Standard Deviation of accuracy scores: {np.std(accuracy_scores)}")

In [13]:
# Using Logistic Regression algorithm for modelling
# TfidfVectorizer for vectorization
# stop words not removed while preprocessing
kf_cv(train_, model = linear_model.LogisticRegression(solver = 'newton-cg', random_state = 42, class_weight = 'balanced'), vectorizer = TfidfVectorizer())

0    346
1    336
4    274
2    273
3    261
Name: Category, dtype: int64
Shape of xtrain: (1192, 22558)
Fold: 0
Accuracy = 0.9731543624161074

Shape of xtrain: (1192, 22283)
Fold: 1
Accuracy = 0.9630872483221476

Shape of xtrain: (1192, 22423)
Fold: 2
Accuracy = 0.9530201342281879

Shape of xtrain: (1192, 22735)
Fold: 3
Accuracy = 0.9697986577181208

Shape of xtrain: (1192, 22484)
Fold: 4
Accuracy = 0.9664429530201343

Mean accuracy score: 0.9651006711409396
Standard Deviation of accuracy scores: 0.006909818886568481


In [14]:
# Using Logistic Regression algorithm for modelling with class weight = 'balanced' to handle class imbalance
# TfidfVectorizer for vectorization
# stop words removed while preprocessing
kf_cv(train_, model = linear_model.LogisticRegression(solver = 'newton-cg', random_state = 42, class_weight = 'balanced'), vectorizer = TfidfVectorizer(), stop_words = set(stopwords.words('english')))

0    346
1    336
4    274
2    273
3    261
Name: Category, dtype: int64
Stop words removed
Shape of xtrain: (1192, 22496)
Fold: 0
Accuracy = 0.9832214765100671

Stop words removed
Shape of xtrain: (1192, 22367)
Fold: 1
Accuracy = 0.9697986577181208

Stop words removed
Shape of xtrain: (1192, 22368)
Fold: 2
Accuracy = 0.959731543624161

Stop words removed
Shape of xtrain: (1192, 22467)
Fold: 3
Accuracy = 0.9765100671140939

Stop words removed
Shape of xtrain: (1192, 22469)
Fold: 4
Accuracy = 0.9731543624161074

Mean accuracy score: 0.9724832214765101
Standard Deviation of accuracy scores: 0.007769018055563917


Getting marginally better performance after removing stopwords

### Train the model on full train dataset

In [21]:
def preprocess(df, stop_words = None):
    # -- lowercase text
    df['Text'] = df['Text'].apply(lambda x : x.lower())
    # -- tokenize text
    df['Text'] = df['Text'].apply(lambda x : " ".join(word_tokenize(x)))
    if stop_words:
        df['Text'] = df['Text'].apply(lambda x : " ".join([word for word in x.split() if word not in stop_words]))
    return df

In [22]:
train_df = preprocess(train_, stop_words = set(stopwords.words('english')))
train_df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens maj...,business
3,1976,lifestyle governs mobile choice faster better ...,tech
4,917,enron bosses $ 168m payout eighteen former enr...,business


In [23]:
train_df.iloc[0,1]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers battery fraud charges called company whistleblower first witness . cynthia cooper worldcom ex-head internal accounting alerted directors irregular accounting practices us telecoms giant 2002. warnings led collapse firm following discovery $ 11bn ( £5.7bn ) accounting fraud . mr ebbers pleaded guilty charges fraud conspiracy . prosecution lawyers argued mr ebbers orchestrated series accounting tricks worldcom ordering employees hide expenses inflate revenues meet wall street earnings estimates . ms cooper runs consulting business told jury new york wednesday external auditors arthur andersen approved worldcom accounting early 2001 2002. said andersen given green light procedures practices used worldcom . mr ebber lawyers said unaware fraud arguing auditors alert problems . ms cooper also said shareholder meetings mr ebbers often passed technical questions company finance chief giving brief answers .

In [24]:
test_df = preprocess(test_, stop_words = set(stopwords.words('english')))
test_df.head()

,ArticleId,Text
0,1018,qpr keeper day heads preston queens park range...
1,1319,software watching work software monitor every ...
2,1138,arcy injury adds ireland woe gordon arcy ruled...
3,459,india reliance family feud heats ongoing publi...
4,1020,boro suffer morrison injury blow middlesbrough...


In [26]:
# fit vectorizer on the training data
vectorizer = TfidfVectorizer()
vectorizer.fit(train_df['Text'])

TfidfVectorizer()

In [27]:
# transform training and test data
xtrain = vectorizer.transform(train_df['Text'])
xtest = vectorizer.transform(test_df['Text'])

In [30]:
ytrain = train_df['Category']

In [31]:
model = linear_model.LogisticRegression(solver = 'newton-cg', random_state = 42, class_weight = 'balanced')
model.fit(xtrain, ytrain)

LogisticRegression(class_weight='balanced', random_state=42, solver='newton-cg')

In [32]:
test_pred = model.predict(xtest)

In [35]:
test_['Category'] = pd.Series(test_pred)

In [36]:
test_.head()

,ArticleId,Text,Category
0,1018,qpr keeper day heads preston queens park range...,sport
1,1319,software watching work software monitor every ...,tech
2,1138,arcy injury adds ireland woe gordon arcy ruled...,sport
3,459,india reliance family feud heats ongoing publi...,business
4,1020,boro suffer morrison injury blow middlesbrough...,sport


In [37]:
test_.to_excel("test.xlsx")